## Clustering: k-means and linkage-based clustering

In this notebook we are going to practice with the k-means and the linkage-based (called "agglomerative" in scikit learn) clustering algorithms.

In particular you are going to implement the k-means algorithm from scratch and to compare the results with the implementation already present in the sklearn library.

In [ ]:
# Load the required packages
# If a package is missing in your setup, install it with 'conda install <package_name>' 
# or with 'pip install <package_name>'
%matplotlib inline  

import numpy as np
import scipy as sp
import imageio as imio
import matplotlib.pyplot as plt
import math
import pandas as pd

import sklearn
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans
from sklearn.datasets import load_sample_image
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean

from mpl_toolkits.mplot3d import Axes3D #3d plotting functions
from matplotlib import pyplot
from PIL import Image

from copy import deepcopy  #deepcopy ensures that a copy of all the object data is performed (not just the pointers)

print ('scikit-learn version: ', sklearn.__version__)

## TO DO:
    
Place your ID number in the ID variable, it will be used as random seed (as usual the random seed can affect a little bit the results, try to change it)

In [ ]:
# fix your ID ("numero di matricola") and the seed for random generator
ID =   1232236
np.random.seed(ID)

In [ ]:
# load the provided images and display them (if you like you can experiment with other images)
image1 = imio.imread('data/santaclaus2.jpg')
image2  = imio.imread("data/landscape.jpg")
image3  = imio.imread("data/reindeer.jpg")

ax = plt.axes(xticks=[], yticks=[])
ax.imshow(image1)
plt.show()
print("Santa Claus image: ",image1.shape)

ax2 = plt.axes(xticks=[], yticks=[])
ax2.imshow(image2)
plt.show()
print("Landscape image: ",image2.shape)

ax3 = plt.axes(xticks=[], yticks=[])
ax3.imshow(image3)
plt.show()
print("Reindeer image: ",image3.shape)

We are going to start by using the Santa Claus image.


In [ ]:
# reshape the data to a matrix of num_pixels x 3 
# (divide by 255 to have colors in [0 1] range for plotting functions of sklearn)
data = image1.reshape(image1.shape[0]*image1.shape[1], 3)/255
new_img = data.reshape(image1.shape)

print(data.shape)
print(data)

In [ ]:
# Plot the points in the 3-dimensional space with normalized intervals between 0 and 1
# (corresponding to the three channels of the image, i.e. Red Green and Blue)

fig = pyplot.figure()
axis = fig.add_subplot(1, 1, 1, projection="3d")
r, g, b = list(data[:,0]), list(data[:,1]), list(data[:,2])

axis.scatter(r, g, b, c=data, s=5, marker="o")
axis.set_xlabel("Red")
axis.set_ylabel("Green")
axis.set_zlabel("Blue")
pyplot.show()

## TO DO 1
Implement the k-means algorithm manually (**do not use the kmeans function of sklearn and do not download implementations from other web sources**). The inputs to the function is the set of vectors to be clustered and the number of clusters. The output must contain the clusters barycenters, a vector associating each data point to the corresponding cluster and the error (value of the cost function) at each iteration.
Additionally, fix a maximum number of iterations of the k-means algorithm (e.g., 50).

Be careful about the initalization, you can use some random points from the training set, or get random values but ensure they are in the proper range. Poor initalizations can lead to the failure of the algorithm (in particular check that no cluster is initialized as empty, otherwise the algorithm can not update it).

In [ ]:
def distance(x,y):
    return math.sqrt(sum([(x[i]-y[i])**2 for i in range(len(x))]))

max_iters=50

def my_kmeans(points, k):
    n = 0

    points = pd.DataFrame(points,columns=['r','g','b'])
    
    # Initialization
    centroids = np.random.random((k,3))
    old_centroids = np.zeros((k,3))
    error = [0]*max_iters
    clusters = 0
    
    # Update rule
    while n < max_iters:
        
        cost = 0
        
        for cl in range(k):                
                points[cl]=((points['r']-centroids[cl][0])**2+(points['g']-centroids[cl][1])**2+(points['b']-centroids[cl][2])**2).pow(1./2)
        
        points['cluster'] = points[[c for c in range(k)]].idxmin(axis=1)
        
        # Recalculating centroids
        for cl in range(k):
            centroids[cl] = points.loc[points['cluster']==cl,['r','g','b']].mean()

        # Computing cost function
        conditions = [points['cluster']==cl for cl in range(k)]
        choices = [points[cl] for cl in range(k)]
        points['cost'] = np.select(conditions, choices)
        points['cost'] = points['cost'].pow(2)
        cost = sum(points['cost'])
        error[n-1] = cost
        
        # Convergence check
        if np.array_equal(centroids, old_centroids): 
            print('Convergence reached with ', n, ' iterations')
            break
        old_centroids = centroids.copy()
        
        n+=1
    
    clusters = points['cluster']
    
    return centroids, clusters, error

## TO DO 2:

Now try the function you developed on the Santa Claus image with three clusters (k=3). 

Then plot the data points in the 3-dimensional space, each point must be coloured based on the membership to one of the clusters. Additionally, plot the respective clusters centroids (use a different shape, size or color to highlight the centroids).

In [ ]:
mykmeans_centers,clusters,error =  my_kmeans(data,3)

print('Centroids:')
print(mykmeans_centers)

fig = pyplot.figure()
axis = fig.add_subplot(1, 1, 1, projection="3d")
axis.set_xlabel("Red")
axis.set_ylabel("Green")
axis.set_zlabel("Blue")
axis.scatter(r, g, b, marker="o", c=clusters, s=1, cmap='viridis', zorder=0, alpha=0.5 )
axis.scatter(mykmeans_centers[:,0], mykmeans_centers[:,1], mykmeans_centers[:,2], c='black', s=400, zorder=10)
pyplot.show()

As can be seen by executing several times the cell above, the final result strongly depends on the initial (random) centroids. So it can be useful to repeat the execution to be more sure about the results.

### TO DO 3: 
Plot the value of the error versus the number of iterations

In [ ]:
_, ax = plt.subplots(1,1,figsize=(8,8))
ax.set_xlabel('Number of iterations')
ax.set_ylabel('Error')
ax.set_title('Variation of the cost function')
ax.plot(np.arange(len(error)),error)

### TO DO 4:
Now use the k-means function provided in sklearn. Pass to the function the number of clusters and use multiple random initializations (n_init parameter). Go to the documentation page for further details

In [ ]:
kmeans =  KMeans(n_clusters=3,init='random')
kmeans.fit(data)
print(kmeans.cluster_centers_)
print(kmeans.labels_)
print(kmeans.inertia_)

### TO DO 5:
Perform the same plot as above but with the output of the k-means function provided in sklearn.

In [ ]:
ax1 = plt.figure(figsize=(8,8)).add_subplot(1,1,1,projection='3d')

# RGB plot
ax1.set_xlabel("Red")
ax1.set_ylabel("Green")
ax1.set_zlabel("Blue")
ax1.set_title("Cluster with sklearn")
ax1.scatter(r, g, b, marker="o", c=kmeans.labels_, s=1, cmap='viridis', zorder=0, alpha=0.5 )
ax1.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], kmeans.cluster_centers_[:,2], c='black', s=400, zorder=10)
pyplot.show()

## Question 1: 

Compare the results obtained with your implementation and with k-means from sklearn. Do you observe any differences, i.e., do the two plots match? 

The plots obtained are almost equivalent, and also the numeric values of the centroids are very similar to each other, sign of the fact that the manual algorithm has been implemented correctly. Also repeating the execution of the algorithm we keep obtaining the same results. Looking at the behaviour of the cost function we find what we were expecting: the error decrease after each iteration until the complete convergence. <br>
Something that is interesting to notice is that, even the algorithm find alway the same centroids, the color to which they are associated depends on the initial values randomly sampled.

### TO DO 6:

Now display the segmented image based on the 3 clusters found above with both the k-means functions by sklearn and your k-means implementation

In [ ]:
_,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(18,6))

ax1.imshow(image1)
ax1.set_title('Original Image')

# SK learn 
sk_clustered_santa = kmeans.cluster_centers_[kmeans.labels_]
sk_clustered_santa = sk_clustered_santa.reshape(image1.shape)
ax2.imshow(sk_clustered_santa)
ax2.set_title('SK learn - clustering')

# Personal implementation
ps_clustered_santa = mykmeans_centers[clusters]
ps_clustered_santa = sk_clustered_santa.reshape(image1.shape)
ax3.imshow(ps_clustered_santa)
ax3.set_title('Personal - clustering')

## Question 2: 

What do you observe? Do you think clustering is useful for image segmenation? And for image compression?  Comment your answer.

I observe an image that is decomposed into its base 3 colors (red, white and green). I think that clustering is useful for image segmentation since it allow us to highlight image contours (at least when these are well defined). Its usage for image compression, instead, depends on the image itself, in particular if we can still recognize it after the process.

## TO DO 8:

Now load the landscape image (optional: try also with the reindeer image) and segment it using kmeans with k varying from 2 to 15 clusters. You can use the sklearn implementation.

Then plot the resulting data points in the 3-dimensional space, each point must be colored based on the cluster membership. 
Additionally, plot the respective clusters centroids.



In [ ]:
image = image2    # LANDSCAPE
#image = image3    # REINDEER
data = image.reshape(image.shape[0]*image.shape[1], 3) / 255
print(data.shape)
r, g, b = list(data[:,0]), list(data[:,1]), list(data[:,2])
errors = [0]*14

for k in range(2,16):
    kmeans =  KMeans(n_clusters=k,init='random')
    kmeans.fit(data)
    fig = plt.figure(figsize=(15,7))
    ax1 = fig.add_subplot(1,2,1,projection="3d")
    ax2 = fig.add_subplot(1,2,2)
    ax1.set_xlabel("Red")
    ax1.set_ylabel("Green")
    ax1.set_zlabel("Blue")
    ax1.set_title("Cluster with sklearn")
    ax1.scatter(r, g, b, marker="o", c=kmeans.labels_, s=1, cmap='viridis', zorder=0, alpha=0.5 )
    ax1.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], kmeans.cluster_centers_[:,2], c='black', s=400, zorder=10)
    errors[k-2] = kmeans.inertia_
    
    # Clustered image
    clustered_image = kmeans.cluster_centers_[kmeans.labels_]
    clustered_image = clustered_image.reshape(image.shape)
    ax2.imshow(clustered_image)
    ax2.set_title('Clustered image')

## TO DO 9:

Plot for different values of k (e.g. k between 2 and 15) the respective error of the kmeans algorithm 

In [ ]:
_, ax = plt.subplots(1,1,figsize=(8,8))
ax.set_xlabel('Number of clusters')
ax.set_ylabel('Error')
ax.set_title('Variation of the cost function respect to the number of clusters')
ax.plot(np.arange(len(errors)),errors)

##  Question 3:

Compare the results with different values of k, what do you observe? 

Analyze also the error, which one do you think is the optimal value of k ?

Is there a single, clear answer ? 

As can be seen in the plot, the error decrease at the increasing of the number of clusters used in the process: this was predictable, since we are using each time a more precise analysis instrument. I think that there isn't an optimal value of k, since, due to overfitting, the biggest is k, the smallest will be the error. <br>
The "perfect" value of k is the number of the pixel of the image: in that case we would have the maximum precision possible. But in that case we would get a dendogram, that is useless for our pourpose.

## Linkage-based clustering

The second part of the assignment concern instead linkage-based clustering. We will use the AgglomerativeClustering module of sklearn. 

In [ ]:
# Import required packages
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics, datasets
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler


# Load sample data
data = np.load('data/moon_data.npz')
print(data.files)
X = data['X']
labels_true = data['labels_true']

### TO DO 10: 

Now exploit the AgglomerativeClustering algorithm on the provided sample data points. Use the "single" linkage type that correspond to the minimum distance criteria seen in the lectures and 2 clusters. Notice that the "single" option has been introduced recently in sklearn, if you get an error ensure you have a recent version of the library. Plot the resulting clustering.

In [ ]:
# #############################################################################
# Compute Agglomerative Clustering

# Standarize features
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

db = AgglomerativeClustering(n_clusters=2,linkage="single")
db.fit(X_std) 

In [ ]:
# Plot result

fig = pyplot.figure(figsize=(15,6))
ax1 = fig.add_subplot(1, 2, 1, projection="3d")
r, g, b = list(X[:,0]), list(X[:,1]), list(labels_true)

ax1.scatter(r, g, b, c=labels_true, s=5, marker="o")
ax1.set_xlabel("Red")
ax1.set_ylabel("Green")
ax1.set_zlabel("Blue")
ax1.set_title("Original RGB image")

ax2 = fig.add_subplot(1, 2, 2, projection="3d")
ax2.scatter(r, g, b, marker="o", c=db.labels_, s=1, cmap='viridis', zorder=0, alpha=0.5 )
ax2.set_xlabel("Red")
ax2.set_ylabel("Green")
ax2.set_zlabel("Blue")
ax2.set_title("Clustered RGB image")

pyplot.show()

### TO DO 11: 

Now try the KMeans with two clusters on the same dataset we used for the AgglomerativeClustering algorithm.

In [ ]:
kmeans =  KMeans(n_clusters=2,init='random')
kmeans.fit(X_std)

ax = plt.figure(figsize=(8,8)).add_subplot(1,1,1,projection='3d')

# RGB plot
ax.set_xlabel("Red")
ax.set_ylabel("Green")
ax.set_zlabel("Blue")
ax.set_title("Cluster with sklearn")
ax.scatter(r, g, b, marker="o", c=kmeans.labels_, s=1, cmap='viridis', zorder=0, alpha=0.5 )
ax.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], c='black', s=400, zorder=10)
pyplot.show()

## Question 4:

Compare the results of K-means and Agglomerative Clustering and explain what you observe and why?



There is an obvious distance between the two methods: while agglomerative clustering group the points in a way that is visibly "more correct", the k means algorithm seems to "act" better in the vertical dimension, since it divide the points into two clusters basing on their relative distances and don't taking into account the fact that they are located into different plains. This phenomenon is due to the research of a spherycal n-dimensional symmetry that the k-means wants to find.